In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image

In [ ]:
#!/usr/bin/env python3
import os
import sys
import threading
import cv2
import numpy as np
import yaml
import pickle
import pdb

In [ ]:
model = models.mobilenet_v2(pretrained=True)
model.classifier = nn.Identity()

In [ ]:
model

In [ ]:
def get_features(model, image_pil):
    scaler = transforms.Resize((224, 224))
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    to_tensor = transforms.ToTensor()
    t_img = Variable(normalize(to_tensor(scaler(image_pil))).unsqueeze(0))
    foo = model(t_img)
    return foo

In [ ]:
def get_cnn_features_from_video(downsampled_video_filename, video_name, keyframe_interval):
    "Receives filename of downsampled video and of output path for features. Extracts features in the given keyframe_interval. Saves features in pickled file."

    images = get_keyframes(downsampled_video_filename, keyframe_interval)
    data = []
    counter = 0
    for image in images:
        image_cv = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        image_pil = Image.fromarray(image_cv)
#        cv2.imwrite('frame.jpg', image)
        features = get_features(model, image_pil)
        data.append(features)
    print(downsampled_video_filename)
    try:
        data = np.array(data)
    except:
        #continue
        pass
        
#    if data is not []:
    pickle.dump(data, open(str('cnn/'+video_name+'.pkl'), 'wb'))
    #np.savez('cnn/'+video_name+'.npz',data)

In [ ]:
def get_keyframes(downsampled_video_filename, keyframe_interval):
    "Generator function which returns the next keyframe."

    # Create video capture object
    video_cap = cv2.VideoCapture(downsampled_video_filename)
    frame = 0
    while True:
        frame += 1
        ret, img = video_cap.read()
        if ret is False:
            break
        if frame % keyframe_interval == 0:
            yield img
    video_cap.release()

In [ ]:
#python surf_feat_extraction.py -i list/all.video config.yaml
all_video_names = 'list/all.video'#sys.argv[1]
config_file = 'config.yaml'#sys.argv[2]
my_params = yaml.load(open(config_file))

In [ ]:
# Get parameters from config file
keyframe_interval = my_params.get('keyframe_interval')
hessian_threshold = my_params.get('hessian_threshold')
# surf_features_folderpath = my_params.get('surf_features')
downsampled_videos = my_params.get('downsampled_videos')

In [ ]:
fread = open(all_video_names, "r")
for line in fread.readlines():
    video_name = line.replace('\n', '')
    downsampled_video_filename = os.path.join(downsampled_videos, video_name + '.ds.mp4')
#     surf_feat_video_filename = os.path.join(surf_features_folderpath, video_name + '.surf')

    if not os.path.isfile(downsampled_video_filename):
        continue

    # Get SURF features for one video
    get_cnn_features_from_video(downsampled_video_filename,
                                 video_name, keyframe_interval)


In [ ]:
foo = get_keyframes('./downsampled_videos/HVC1012.ds.mp4', keyframe_interval)

In [ ]:
for img in foo:
    print(img)